### Configuration

In [1]:
import os
import re
import numpy as np
import pandas as pd
import datetime

import mne
from scipy.stats import zscore
from tqdm import tqdm

from utils__helpers_macro import hilbert_powerphase, hilbert_envelope
from utils__helpers_epoch import epoch_sw, epoch_sw_2, epoch_spikes
import utils__config

In [2]:

os.chdir(utils__config.working_directory)
os.getcwd()

'g:\\My Drive\\Residency\\Research\\Lab - Damisah\\Project - Sleep'

### Parameters

In [3]:
fif_path = 'Cache/Subject01/S01_Feb02_256hz.fif'
bad_channel_path = 'Cache/Subject01/S01_bad_channels.csv'
hypno_path = 'Cache/Subject01/S01_hypnogram.csv' 
spike_path = 'Cache/Subject01/S01_spikes.csv'
spike_out_path = 'Cache/Subject01/S01_spike_epochs_30s.csv'
hypno_out_path = 'Cache/Subject01/S01_hypno_epochs_30s.csv' 

# fif_path = 'Cache/Subject02/Apr26/S02_Apr26_256hz.fif'
# bad_channel_path = 'Cache/Subject02/Apr26/S02_bad_channels.csv'
# hypno_path = 'Cache/Subject02/Apr26/S02_hypnogram.csv'
# spike_path = 'Cache/Subject02/Apr26/S02_spikes.csv'
# spike_out_path = 'Cache/Subject02/Apr26/S02_spike_epochs_30s.csv'
# hypno_out_path = 'Cache/Subject02/Apr26/S02_hypno_epochs_30s.csv'

# fif_path = 'Cache/Subject02/Apr27/S02_Apr27_256hz.fif'
# bad_channel_path = 'Cache/Subject02/Apr27/S02_bad_channels.csv'
# hypno_path = 'Cache/Subject02/Apr27/S02_hypnogram.csv'
# spike_path = 'Cache/Subject02/Apr27/S02_spikes.csv'
# spike_out_path = 'Cache/Subject02/Apr27/S02_spike_epochs_30s.csv'
# hypno_out_path = 'Cache/Subject02/Apr27/S02_hypno_epochs_30s.csv'

In [4]:
epoch_length = 30 # bin width in seconds
sampling_freq = 256 # (s.f. used to detect slow waves)
hypno_sfreq = 256 # (s.f. used to make hypnogram)
tmin = 'none' # datetime.datetime(2022, 4, 28, 0, 0, 0, 0, tzinfo = datetime.timezone.utc)
tmax = 'none'

### Load Data

In [5]:
raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)

# Select only macroelectrodes
raw.pick_types(seeg = True, ecog = True)

# Remove rejected channels
bad_channels = pd.read_csv(bad_channel_path)
bad_channels = bad_channels[bad_channels['channel'].isin(raw.ch_names)]
raw.drop_channels(ch_names = bad_channels['channel'].astype('string'))

# Set the bin size; we set the stop interval
# to the total length of the recording
if tmin != 'none':

    last_bin = int((tmax - tmin).seconds / epoch_length) # will use this later
    bin_list = np.arange(0, (tmax - tmin).seconds + 1, epoch_length)

# Define tmin/tmax as the start/end of recording if not specified
else:
    
    tmin = raw.times[0]
    tmax = raw.times[-1]

    last_bin = int((tmax - tmin) / epoch_length) # will use this later
    bin_list = np.arange(0, (tmax - tmin) + 1, epoch_length)

Opening raw data file Cache/Subject01/S01_Feb02_256hz.fif...


C:\Users\Layton\AppData\Local\Temp\ipykernel_20200\1349684526.py:1: RuntimeWarning: This filename (Cache/Subject01/S01_Feb02_256hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)


Isotrak not found
    Range : 0 ... 1843573 =      0.000 ...  7201.457 secs
Ready.
Reading 0 ... 1843573  =      0.000 ...  7201.457 secs...


### Bin Hypnogram into Epochs

In [6]:
hypno = pd.read_csv(hypno_path, header = None)
hypno = hypno.reset_index()
hypno.columns = ['idx', 'stage']

# Bin the hypnogram into epochs based on sample number
hypno['epoch'] = pd.cut(hypno['idx'], bins = bin_list * hypno_sfreq, labels = False, include_lowest = True)

# Select the mode of the sleep stage within each epoch
hypno = hypno.groupby(['epoch'])['stage'].agg(pd.Series.mode).reset_index()

hypno.to_csv(hypno_out_path, index = False)

### Bin spikes into FR per Epoch

In [7]:
spikes = epoch_spikes(spike_path = spike_path, 
                      bin_width = epoch_length, 
                      bin_list = bin_list, 
                      last_bin = last_bin)

spikes.to_csv(spike_out_path, index = False)